In [17]:
import os
import re
from google.cloud import vision
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'config/config_ocr.json'

mime_type = 'application/pdf'
batch_size = 2
client = vision.ImageAnnotatorClient()

feature = vision.Feature(
type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

gcs_source_uri = 'gs://chatbot-mango.appspot.com/1.pdf'
gsc_destination_uri = 'gs://chatbot-mango.appspot.com/result'
gcs_source = vision.GcsSource(uri=gcs_source_uri)
input_config = vision.InputConfig(
gcs_source=gcs_source, mime_type=mime_type)

gcs_destination = vision.GcsDestination(uri=gsc_destination_uri)
output_config = vision.OutputConfig(
gcs_destination=gcs_destination, batch_size=batch_size)

async_request = vision.AsyncAnnotateFileRequest(
features=[feature], input_config=input_config,
output_config=output_config)

operation = client.async_batch_annotate_files(
requests=[async_request])

print('Waiting for the operation to finish.')
operation.result(timeout=420)
storage_client = storage.Client()

match = re.match(r'gs://([^/]+)/(.+)', gsc_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)

bucket = storage_client.get_bucket(bucket_name)

# List objects with the given prefix.
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)

Waiting for the operation to finish.
Output files:
resultoutput-1-to-1.json
